### Машинное обучение
## Домашнее задание №3 - Градиентный бустинг

**Общая информация**

**Срок сдачи:** 14 мая 2024, 23:59   
**Штраф за опоздание:** -2 балла за каждые сутки

Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Считаем производные для функций потерь (1 балл)

Мы будем реализовать градиентный бустинг для 3 функций потерь:

1) MSE  $L(a(x_i), y_i) = (y_i - a(x_i)) ^ 2$

2) Экспоненциальная  $L(a(x_i), y_i) = exp( -a(x_i) y_i), y_i \in \{-1, 1\}$

3) Логистическая  $L(a(x_i), y_i) = \log (1 + exp( -a(x_i) y_i)), y_i \in \{-1, 1\}$

где $a(x_i)$ предсказание бустинга на итом объекте. 

Для каждой функции потерь напишите таргет, на который будет настраиваться каждое дерево в бустинге. 

Ваше решение тут

1) MSE  $𝐿(𝑎(𝑥_𝑖),𝑦_𝑖)= 2(𝑦_𝑖−𝑎(𝑥_𝑖))$

2) Экспоненциальная  $L(a(x_i), y_i) = y_i*exp( -a(x_i) y_i), y_i \in \{-1, 1\}$

3) Логистическая  $L(a(x_i), y_i) = y_i / (exp(a(x_i)y_i) + 1)$

##  Реализуем градиентный бустинг (3 балла)

Реализуйте класс градиентного бустинга для классификации. Ваша реализация бустинга должна работать по точности не более чем на 5 процентов хуже чем GradientBoostingClassifier из sklearn. 


Детали реализации:

-- должно поддерживаться 3 функции потерь

-- сами базовые алгоритмы(деревья, линейные модели и тп) реализовать не надо, просто возьмите готовые из sklearn

-- в качестве функции потерь для построения одного дерева используйте MSE

-- шаг в бустинге можно не подбирать, можно брать константный

-- можно брать разные модели в качестве инициализации бустинга

-- должны поддерживаться следующие параметры:

а) число итераций
б) размер шага
в) процент случайных фичей при построении одного дерева
д) процент случайных объектов при построении одного дерева
е) параметры базового алгоритма (передавайте через **kwargs)

In [4]:
import numpy as np

from sklearn.datasets import load_wine
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [27]:
class MyGradientBoostingClassifier:

    def __init__(self, loss='log', learning_rate=0.1, n_estimators=100, colsample=1.0, subsample=1.0, *args, **kwargs):
        """
        loss -- один из 3 лоссов:
        learning_rate -- шаг бустинга
        n_estimators -- число итераций
        colsample -- процент рандомных признаков при обучнеии одного алгоритма
        colsample -- процент рандомных объектов при обучнеии одного алгоритма
        args, kwargs -- параметры  базовых моделей
        """
        self.loss = loss
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.colsample = colsample
        self.subsample = subsample
        self.model_params = kwargs
        
    
    def fit(self, X, y, base_model=DecisionTreeRegressor, init_model=None):
        """
        X -- объекты для обучения:
        y -- таргеты для обучения
        base_model -- класс базовых моделей, например sklearn.tree.DecisionTreeRegressor
        init_model -- класс для первой модели, если None то берем константу (только для посл задания)
        """
        if not init_model:
            self.first_model = False
            y_pred = np.full(X.shape[0], np.mean(y))
            self.ensemble = [np.mean(y)]
        else:
            self.first_model = True
            pass
        for i in range(self.n_estimators):
            if i > 0:
                y_pred += self.learning_rate * self.ensemble[i].predict(X)
            if self.subsample == 1.0:
                X_train = X
                y_train = y_pred
            else:
                X_train, _, y_train, _ = train_test_split(X, y_pred, train_size=self.subsample)
            if self.loss == 'mse':
                y_train = 2 * (y - y_train)
            elif self.loss == 'exp':
                y_train = y * np.exp(-y_train * y)
            elif self.loss == 'log':
                y_train = y / (1 + np.exp(y_train * y))
            current_model = base_model(**self.model_params)
            current_model.fit(X_train, y_train)
            self.ensemble.append(current_model)
        
        
    def predict(self, X):
        if self.first_model:
            pass
        else:
            y_pred = np.full(X.shape[0], self.ensemble[0])
        for i in range(1, len(self.ensemble)):
            y_pred += self.learning_rate * self.ensemble[i].predict(X)
        self.margins = y_pred
        return np.sign(y_pred)
    
    
    def get_margins(self):
        return self.margins


In [28]:
my_clf_0 = MyGradientBoostingClassifier()
my_clf_1 = MyGradientBoostingClassifier()
my_clf_2 = MyGradientBoostingClassifier()

clf = GradientBoostingClassifier()

In [20]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)
y_train_0 = np.copy(y_train)
y_train_0[np.where((y_train == 1) | (y_train == 2))] = -1
y_train_0[np.where(y_train == 0)] = 1

y_train_1 = np.copy(y_train)
y_train_1[np.where((y_train == 0) | (y_train == 2))] = -1
y_train_1[np.where(y_train == 1)] = 1

y_train_2 = np.copy(y_train)
y_train_2[np.where((y_train == 1) | (y_train == 0))] = -1
y_train_2[np.where(y_train == 2)] = 1

In [43]:
my_clf_0.fit(X_train, y_train_0)
my_clf_1.fit(X_train, y_train_1)
my_clf_2.fit(X_train, y_train_2)
my_clf_0.predict(X_test)
my_clf_1.predict(X_test)
my_clf_2.predict(X_test)

margin_0 = my_clf_0.get_margins()
margin_1 = my_clf_1.get_margins()
margin_2 = my_clf_2.get_margins()
margin = np.vstack((margin_0, margin_1, margin_2)).T

ind = np.argmax(margin, axis=1)

clf.fit(X_train, y_train)
print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))
print(accuracy_score(y_pred=ind, y_true=y_test))

0.9444444444444444
0.9444444444444444


## Подбираем параметры (2 балла)

Давайте попробуем применить Ваш бустинг для предсказаний цены домов в Калифорнии. Чтобы можно было попробовтаь разные функции потерь, переведем по порогу таргет в 2 класса: дорогие и дешевые дома.

В задании нужно

1) Построить график точности в зависимости от числа итераций на валидации.

2) Подобрать оптимальные параметры Вашего бустинга на валидации. 


In [28]:
from sklearn.datasets import fetch_california_housing
X, y = fetch_california_housing(return_X_y=True)

In [26]:
# Превращаем регрессию в классификацию
y = (y > 2.0).astype(int)
print(X.shape, y.shape)

(20640, 8) (20640,)


## BooBag BagBoo (1 балл)



Попробуем объединить бустинг и бэгинг. Давайте

1) в качестве базовой модели брать не дерево решений, а случайный лес (из sklearn)

2) обучать N бустингов на бустрапированной выборке, а затем предикт усреднять

Попробуйте обе этих стратегии на данных из прошлого задания. Получилось ли улучшить качество? Почему?

## Умная инициализация (1 балл)

Попробуйте брать в качестве инициализации бустинга не константу, а какой-то алгоритм и уже от его предикта стартовать итерации бустинга. Попробуйте разные модели из sklearn: линейные модели, рандом форест, svm..

Получилось ли улучшить качество? Почему?



## Фидбек (бесценно)

* Какие аспекты обучения  ансамблей Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

